In [1]:
import pandas as pd
import re
import PyPDF2
import os
import json
import copy
import numpy as np
import tabula as tb

In [2]:
pdf_path = "2015_house_pdfs/Pelosi_Nancy_10010857.pdf"
pdf_path_joe = "2015_house_pdfs/Wilson_Joe_10012512.pdf"
pdf_to_empty = "2015_house_pdfs/Pelosi_Nancy_20002351.pdf"
pdf_to_apple = "2018_house_pdfs/Apple_Gary Winston_10022824.pdf"
pdf_to_young = "2017_house_pdfs/Young_Stephen Robert Neale_10018941.pdf"
pdf_to_wass = "2019_house_pdfs/Wasserman Schultz_Debbie_10037015.pdf"
pdf_to_nancy = "2019_house_pdfs/Pelosi_Nancy_10035243.pdf"

In [3]:
pdf_to_empty = "2015_house_pdfs/Pelosi_Nancy_20002351.pdf"
if os.stat(pdf_to_empty).st_size < 2000:
    print("smaller")
else:
    print("larger")

smaller


In [4]:
def create_template(file_path):
    file = open(file_path, 'rb')
    readpdf = PyPDF2.PdfFileReader(file)
    total_pages = readpdf.numPages
    with open("pdf_reduced_template.tabula-template.json", "r") as jsonTemplate:
        data = json.load(jsonTemplate)
        page_template = data[3]
        # print(page_template)
        if total_pages == 1:
            data = data[:-1]
        else:
            for page_num in range(2, total_pages):
                tmp_page_template = copy.copy(page_template)
                tmp_page_template['page'] = page_num + 1
                # print(tmp_page_template)
                data.append(tmp_page_template)
            
        with open("pdf_tmp_template.tabula-template.json", 'w') as outfile:
            json.dump(data, outfile)
                
            
            

In [5]:
def scrape_pdf(file):
    create_template(file)
    template_path = 'pdf_tmp_template.tabula-template.json'
    return tb.read_pdf_with_template(file, template_path, stream=True)

In [6]:
db = scrape_pdf(pdf_path)

In [7]:
db_joe = scrape_pdf(pdf_path_joe)

In [8]:
db_apple = scrape_pdf(pdf_to_apple)

In [9]:
db_young = scrape_pdf(pdf_to_young)

In [10]:
db_wass = scrape_pdf(pdf_to_wass)

In [11]:
db_nancy = scrape_pdf(pdf_to_nancy)

In [12]:
pdf_db = pd.DataFrame(columns=['Member Name', 'Member District', 'Company', 'Ticker', 'Type', 'Date',
                      'Value Lower Bound','Value Upper Bound', 'Description', 'Link'])

In [13]:
db[7]

,asset owner,value of asset,income,Unnamed: 0,income tx. >
0,NaN,NaN,type(s),NaN,"$1,000?"
1,NaN,NaN,NaN,NaN,cdfeg
2,Account,NaN,NaN,NaN,NaN
3,"DESCRIPTION: San Francisco, CA",NaN,NaN,NaN,NaN
4,Union Bank of California - Pelosi Trust Checki...,"$1 - $1,000",None,NaN,ecgdf
5,Account,NaN,NaN,NaN,NaN
6,"DESCRIPTION: San Francisco, CA",NaN,NaN,NaN,NaN
7,United Football League SP,"$5,000,001 -",None,NaN,fcedg
8,NaN,"$25,000,000",NaN,NaN,NaN
9,"LOCATION: Jacksonville/Duval, FL, US",NaN,NaN,NaN,NaN


In [14]:
"schedule b: transactions" in db[7]["asset owner"].str.lower().values

True

In [15]:
db[8].iloc[:, 0]

0                                                   NaN
1                                                   NaN
2                              Salesforce.com Inc (CRM)
3                                                   NaN
4     DESCRIPTION: Contribution of 1,084 shares held...
5                                             accounts.
6                                         Visa Inc. (V)
7                                                   NaN
8                     DESCRIPTION: Sale of 5,000 Shares
9                                         Visa Inc. (V)
10                                                  NaN
11    DESCRIPTION: Contribution of 2,000 shares held...
12                                            accounts.
13                            ScHeDule c: earneD income
14                                          Source type
15             Financial Leasing Services Spouse Salary
16           Fairfield Associates Spouse Management Fee
17    Forty-Five Belden Corporation Spouse Manag

In [16]:
index = db[6].index[db[6]["asset owner"].str.lower() == "schedule b: transactions"].tolist()
index



[]

In [17]:
def find_start(df):
    for i in range(len(df)):
        index = df[i].index[df[i].iloc[:, 0].str.lower() == "schedule b: transactions"].tolist()
        if index != []:
            return i, index[0]

In [18]:
find_start(db_joe)

(3, 11)

In [19]:
db[0].iloc[2, 1]

'CA12'

In [20]:
pd.to_datetime(db[8].iloc[9, 1])

Timestamp('2015-12-30 00:00:00')

In [22]:
result = re.match(honorifics_regex, 'hon. nancy pelosi')
result[2]+" " +result[3]

'nancy pelosi'

In [93]:
db_nancy[3]

,asset owner,value of asset,Unnamed: 0,income,income.1,tx. >
0,NaN,NaN,NaN,type(s),NaN,"$1,000?"
1,Aristotle LLC [OL] SP,"$15,001 - $50,000",NaN,None,NaN,dcfeg
2,"LOCATION: San Francisco/San Francisco, CA, US",NaN,NaN,NaN,NaN,NaN
3,DESCRIPTION: Ltd. Partnership Investment in Ar...,NaN,NaN,NaN,NaN,NaN
4,AT&T Inc. (T) [ST] SP,"$250,001 -",NaN,Dividends,"$15,001 - $50,000",ecdgf
5,NaN,"$500,000",NaN,NaN,NaN,NaN
6,Auberge du Soleil [OL] SP,"$5,000,001 -",NaN,Partnership Loss,"$5,001 - $15,000",gdecf
7,NaN,"$25,000,000",NaN,NaN,NaN,NaN
8,"LOCATION: Rutherford/Napa, CA, US",NaN,NaN,NaN,NaN,NaN
9,DESCRIPTION: Ltd. Partnership Investment in Re...,NaN,NaN,NaN,NaN,NaN


In [23]:
test = np.nan
test is np.nan

True

In [64]:
def get_column_for_bounds(df, df_index):
    bounds_regex = r"^\$?([\d,]+)\s*-?\s*\$?([\d,]*)$"
    number_of_columns = len(df[df_index].columns)
    number_of_rows = len(df[df_index])
    column_number = 2
    row_number = 0
    bounds_regex_result = re.match(bounds_regex, str(df[df_index].iloc[row_number, column_number]))
    while bounds_regex_result is None and column_number < number_of_columns:
        if row_number < (number_of_rows - 1):
            row_number += 1
        else:
            row_number = 0
            column_number += 1
        
        bounds_regex_result = re.match(bounds_regex, str(df[df_index].iloc[row_number, column_number]))
    
    return column_number
        

In [66]:
db[7]

,asset owner,value of asset,income,Unnamed: 0,income tx. >
0,NaN,NaN,type(s),NaN,"$1,000?"
1,NaN,NaN,NaN,NaN,cdfeg
2,Account,NaN,NaN,NaN,NaN
3,"DESCRIPTION: San Francisco, CA",NaN,NaN,NaN,NaN
4,Union Bank of California - Pelosi Trust Checki...,"$1 - $1,000",None,NaN,ecgdf
5,Account,NaN,NaN,NaN,NaN
6,"DESCRIPTION: San Francisco, CA",NaN,NaN,NaN,NaN
7,United Football League SP,"$5,000,001 -",None,NaN,fcedg
8,NaN,"$25,000,000",NaN,NaN,NaN
9,"LOCATION: Jacksonville/Duval, FL, US",NaN,NaN,NaN,NaN


In [67]:
get_column_for_bounds(db, 7)

4

In [127]:
def generate_entry(df, df_index, row, regex_result, new_db_page):
    
    description_regex = r"^description: (.+)$"
    stock_regex = r"^\s*(?!description:|location:)\s*(.*)\s*\((?!one|two)([\w | .]{1,5})\).*$"
    bounds_regex = r"^[a-zA-Z]*\s*\$?([\d,]+)\s*-?\s*\$?([\d,]*).*$"
    time_regex = r"^[a-zA-z]*\s*(\d{1,2}\/\d{1,2}\/\d{2,4})\s*$"
        
    company_name = regex_result[1].strip()
    ticker = regex_result[2].upper()
    
    time_regex_result = re.match(time_regex, str(df[df_index].iloc[row, 1]))
    print(df[df_index].iloc[row, 0])
    owner_date = pd.to_datetime(time_regex_result[1])
    
    transaction_type = df[df_index].iloc[row, 2]
    column_bounds = get_column_for_bounds(df, df_index)
    bounds_regex_result = re.match(bounds_regex, df[df_index].iloc[row, column_bounds])
    # print(df[df_index].iloc[row, column_bounds])
    lower_bound = bounds_regex_result[1]
    """
    if new_db_page:
        bounds_regex_result = re.match(bounds_regex, df[df_index].iloc[row, 3])
        print(df[df_index].iloc[row, 3])
    else:
        print(df_index, row)
        bounds_regex_result = re.match(bounds_regex, df[df_index].iloc[row, 4])
        print(df[df_index].iloc[row, 3])
    lower_bound = bounds_regex_result[1]
    """
    upper_bound = ''
    
    if bounds_regex_result[2] == '':
        """
        if new_db_page:    
            regex_result = re.match(bounds_regex, df[df_index].iloc[row+1, 3])
            upper_bound = regex_result[1]
        else:
            regex_result = re.match(bounds_regex, df[df_index].iloc[row+1, 4])
            upper_bound = regex_result[1]
        """
        regex_result = re.match(bounds_regex, df[df_index].iloc[row+1, column_bounds])
        upper_bound = regex_result[1]
            
    else:
        upper_bound = bounds_regex_result[2]
    
    
    description = None
        
    row += 1
    
    stock_info = str(df[df_index].iloc[row, 0]).lower()
    
    potential_new_stock = re.match(stock_regex, stock_info)
    exit = True
    while potential_new_stock is None and exit:
        regex_result = re.match(description_regex, str(df[df_index].iloc[row, 0]).lower())
        if regex_result is not None:
            description = regex_result[1]
        if len(df[df_index].index)-1 == row:
            exit = False
        else:
            row += 1
            stock_info = str(df[df_index].iloc[row, 0]).lower()
            potential_new_stock = re.match(stock_regex, stock_info)
        
    return {'Member Name': np.nan,
            'Member District': np.nan,
            'Company': company_name,
            'Ticker': ticker,
            'Type': transaction_type,
            'Date': owner_date,
            'Value Lower Bound' : lower_bound,
            'Value Upper Bound' : upper_bound,
            'Description' : description,
            'Link' : np.nan}, row
        

In [34]:
res = re.match(stock_regex, str(db[7].iloc[31, 0]).lower())
test_entry, row = generate_entry(db, 7, 31, res, False)
test_entry

nan


{'Member Name': nan,
 'Member District': nan,
 'Company': 'hertz global holdings, inc',
 'Ticker': 'HTZ',
 'Type': 'P',
 'Date': Timestamp('2015-07-16 00:00:00'),
 'Value Lower Bound': '100,001',
 'Value Upper Bound': '250,000',
 'Description': 'purchase of 258 call options with a strike price of $14 and an expiration date of 1/15/16',
 'Link': nan}

In [35]:
pdf_db = pd.concat([pdf_db, pd.Series(test_entry).to_frame().T], ignore_index=True)
pdf_db

,Member Name,Member District,Company,Ticker,Type,Date,Value Lower Bound,Value Upper Bound,Description,Link
0,NaN,NaN,"hertz global holdings, inc",HTZ,P,2015-07-16 00:00:00,"100,001","250,000",purchase of 258 call options with a strike pri...,NaN
1,NaN,NaN,"hertz global holdings, inc",HTZ,P,2015-07-16 00:00:00,"100,001","250,000",purchase of 258 call options with a strike pri...,NaN


In [138]:
def process_dataframe(df, link):
    
    pdf_db = pd.DataFrame(columns=['Member Name', 'Member District', 'Company', 'Ticker', 'Type', 'Date',
                      'Value Lower Bound','Value Upper Bound', 'Description', 'Link'])
    
    # description_regex = r"^description: (.+)$"
    stock_regex = r"^\s*(?!description:|location:)\s*(.*)\s*\((?!one|two)([\w | .]{1,5})\).*$"
    # bounds_regex = r"^[a-zA-Z]*\s*\$?([\d,]+)\s*-?\s*\$?([\d,]*).*$"
    honorifics_regex = r"^(hon.?|mr.?|miss|mrs.?|ms.?|dr.?|professor.?|gen.?|)\s*(.+)$"
    # NEED TO ADD TIME REGEX
    
    new_db_page = False

    name_regex_result = re.match(honorifics_regex, df[0].iloc[0, 1].lower())
    name = name_regex_result[2].strip()
                             
    district = df[0].iloc[2, 1]            
    find_start_result = find_start(df)                         
    if find_start_result is None:
        return pdf_db
    
    df_index, row = find_start_result
    
    
    
    row += 1
    stock_info = str(df[df_index].iloc[row, 0]).lower()

    while stock_info != "schedule c: earned income" and stock_info != "schedule d: liabilities":
        regex_result = re.match(stock_regex, stock_info)
        if regex_result is not None:
            result_dict, new_row = generate_entry(df, df_index, row, regex_result, new_db_page)
            result_dict['Member Name'] = name
            result_dict['Member District'] = district
            result_dict['Link'] = link
            pdf_db = pd.concat([pdf_db, pd.Series(result_dict).to_frame().T], ignore_index=True)
            
            
        if len(df[df_index].index)-1 == row:
            row = 0
            df_index += 1
            print("HELLO")
            new_db_page = True
        else:
            row += 1
        # print(stock_info)
        stock_info = str(df[df_index].iloc[row, 0]).lower()
    
    return pdf_db

In [136]:
db_wass[2]

,"ScheDule a: aSSetS anD ""unearneD"" income",Unnamed: 0,Unnamed: 1,Unnamed: 2
0,asset owner value of asset,income,income,tx. >
1,NaN,type(s),NaN,"$1,000?"
2,"""For The Next Generation"" Published by St. Mar...",Royalties,None,egfcd
3,Press [IP],NaN,NaN,NaN
4,"astroNova, Inc. (aLOT) [ST] DC None","Capital Gains,","$2,501 -",cefdg
5,NaN,Dividends,"$5,000",NaN
6,DESCRIPTION: Dependent Child a,NaN,NaN,NaN
7,"astroNova, Inc. (aLOT) [ST] DC None","Capital Gains,","$2,501 -",gdcef
8,NaN,Dividends,"$5,000",NaN
9,DESCRIPTION: Dependent Child B,NaN,NaN,NaN


In [139]:
process_dataframe(db_wass, None)

,Member Name,Member District,Company,Ticker,Type,Date,Value Lower Bound,Value Upper Bound,Description,Link
